# Sentiment Analysis

In this notebook we will show how we build a model to give a polarity to the tweets
which contains about famous comapany

In [3]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import joblib
from collections import Counter
import numpy as np
import pandas as pd
from imblearn.datasets import make_imbalance
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
import re
from langdetect import detect
from bs4 import BeautifulSoup

We have two different dataset of financial tweets to train and test our model, one is downloaded from Kaggle, the other is composed by real tweest screaped from tweeter and where we have put the label of the class by hand.


In [4]:
 # from preprocessing.textCleaner import select_only_keyword, select_only_english, remove_special_char


train_data = pd.read_csv('./data/tweets_with_sentiment.csv')
train_data['text'] = train_data['text'].astype(str)
train_data['text'] = train_data['text'].str.lower()


real_tweets = pd.read_json('../data/train/tweetsAMZN_with_label.json')
#real_tweets = real_tweets.append(pd.read_json('../data/train/tweetsAMZN_with_label.json'), ignore_index=True)
real_tweets.drop(columns=['number_follower', 'number_retweets', 'number_likes', 'number_comments', 'datetime', 'username'], inplace=True)


train_data['text'] = train_data['text'].str.lower()
#train_data = select_only_keyword(real_tweets, 'tesla', 'tsla')
#train_data = select_only_english(real_tweets)
#train_data['text'] = train_data['text'].apply(remove_special_char)

train_data

,text,target
0,video offic mind busi david solomon tell gs in...,neutral
1,price lumber lb f sinc hit ytd high maci turna...,neutral
2,say american dream dead,negative
3,barri silbert extrem optimist bitcoin predict ...,positive
4,satellit avoid attack space junk circl earth paid,negative
...,...,...
28435,fb c f f cb ecf,neutral
28436,btc,neutral
28437,rt hd nuff said tel telcoin telfam crypto bloc...,neutral
28438,btc,neutral


In [5]:
from xgboost import XGBClassifier


#finbert_training_data = finbert.get_data()
x_train, x_test, y_train, y_test = train_test_split(train_data['text'], train_data['target'], test_size=0.2, random_state=11)

print("Before undersampling: ", Counter(y_train))

# Convert x_train to np_array for rebalance
x_train = x_train.values.reshape(-1, 1)
#x_train, y_train = make_imbalance(x_train, y_train,
 #                                 sampling_strategy={'positive': 2000, 'neutral': 2000, 'negative': 2000},
  #                                random_state=0)

# Return to pandas series
x_train = pd.Series(np.squeeze(x_train))
print("After undersampling: ", Counter(y_train))

# Pipeline Classifier
clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', XGBClassifier())   # XGBClassifier è il migliore in acc ma basso positive recall
    # Random Forest Classifier, Logistic Regression e SVC sono buoni (0.75 acc)
    # MultinomialNB e BernoulliNB meno buoni (0.67 acc)
])

# Training the Pipeline Classifier
clf.fit(x_train, y_train)

clf.fit(train_data['text'], train_data['target'])
print(x_train)


#Testing of the Pipeline
predicted = clf.predict(x_test)

#Extracting statistics and metrics
accuracy = accuracy_score(predicted, y_test)
print("Accuracy on test set: ", accuracy)
print("Metrics per class on test set:")

print("Confusion matrix:")
metrics.confusion_matrix(y_test, predicted)

print(metrics.classification_report(y_test, predicted, target_names=["negative", "neutral", "positive"]))

Before undersampling:  Counter({'neutral': 13883, 'positive': 6785, 'negative': 2084})
After undersampling:  Counter({'neutral': 13883, 'positive': 6785, 'negative': 2084})


/Users/tom/.conda/envs/pythonProject/lib/python3.10/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:52:53] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1637426408905/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:52:57] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1637426408905/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0        trade idea mpc realli like one reason free tra...
1                      citrix system ctx set new year high
2        rt kirillklip gem royalti tnrgold tnr v strate...
3                                               rt twitter
4                          inci incyt corpor sec file form
                               ...                        
22747    p

In [6]:
# Save the classifier
joblib.dump(clf, '../model/sentiment_classifier.pkl')

['../model/sentiment_classifier.pkl']

In [7]:
clf = joblib.load('../model/sentiment_classifier.pkl')
my_list = [
"Enameled Cast Iron Shallow Casserole Braiser Pan with Cover, 3.8-Quart, Marine Blue. Deals going from $42.49 to $59.49. Offer ends in 6 day(s) 23 hour(s) 39 minute(s) 4 second(s). Deal type: Best Deal. https://t.co/OZ9xjvg4Gm"

]
arr = np.asarray(my_list)
arr.reshape(-1, 1)
# Use model to make predictions
y_pred = clf.predict(arr)
print(y_pred)

['positive']


In [8]:
clf = joblib.load('../model/sentiment_classifier.pkl')


#Testing with real tweets
real_tweets = pd.read_json('../data/train/tweets_with_label.json')
print("Before undersampling: ", Counter(real_tweets['target']))

# Predicting
predicted = clf.predict(real_tweets['text'].values)

#for i in range(len(predicted)-20,len(predicted)):
 #   print("Predicted: ", predicted[i], " \tReal: ",  real_tweets['target'].values[i])

#Extracting statistics and metrics
accuracy = accuracy_score(real_tweets['target'], predicted)
print("Accuracy on test set: ", accuracy)
print("Metrics per class on test set:")

print("Confusion matrix:")
metrics.confusion_matrix(real_tweets['target'].values, predicted)

print(metrics.classification_report(real_tweets['target'].values, predicted, target_names=["negative", "neutral", "positive"]))

Before undersampling:  Counter({'neutral': 566, 'positive': 143, 'negative': 50})
Accuracy on test set:  0.758893280632411
Metrics per class on test set:
Confusion matrix:
              precision    recall  f1-score   support

    negative       0.71      0.74      0.73        50
     neutral       0.95      0.73      0.83       566
    positive       0.46      0.87      0.60       143

    accuracy                           0.76       759
   macro avg       0.71      0.78      0.72       759
weighted avg       0.84      0.76      0.78       759

